In [ ]:
import http.client
import urllib.parse
import json
import math
import pandas as pd

api_token = "MyapLLgIHDl11lMHGVsPgOhn2apg7fkTl8SWNWnQ"

search_topic = 'crime'
source_domain = 'foxnews.com'
limit = 25

# API Constraint
MAX_RETURN = 20000
max_page = MAX_RETURN // limit

params = urllib.parse.urlencode({
    'api_token': api_token,
    'search': search_topic,
    'domains': source_domain,
    'limit': limit
})


In [20]:
def fetch_news_articles(api_token, search_topic, source_domain, limit):
    """
    Fetch news articles from TheNewsAPI with pagination.
    
    Parameters:
    -----------
    api_token : str
        API authentication token
    search_topic : str
        Search keyword/topic
    source_domain : str
        News source domain (e.g., 'foxnews.com')
    limit : int
        Number of articles per page
        
    Returns:
    --------
    df : pd.DataFrame
        DataFrame containing all fetched articles, sorted by published_at
    """
    # API constraints
    MAX_RETURN = 20000
    max_page = MAX_RETURN // limit
    
    # Build initial params
    params = urllib.parse.urlencode({
        'api_token': api_token,
        'search': search_topic,
        'domains': source_domain,
        'limit': limit
    })
    
    # STEP 1: Initial API request to get metadata (from cell 1)
    content = ""
    try:
        response = requests.get(f"https://api.thenewsapi.com/v1/news/all?{params}")
        raw_content = response.content
        content = raw_content.decode('utf-8')
        parsed_data = json.loads(content)
        print(json.dumps(parsed_data, indent=4))
    except Exception as e:
        print(f"{e}")
        return pd.DataFrame()
    
    # STEP 2: Calculate pagination hyperparameters (from cell 2)
    found = parsed_data['meta']['found']
    found_page = math.ceil(found / limit)
    pages_to_fetch = min(found_page, max_page)
    
    # Fetch last page to get furthest (oldest) news date
    last_page_params = urllib.parse.urlencode({
        'api_token': api_token,
        'search': search_topic,
        'domains': source_domain,
        'limit': limit,
        'page': found_page
    })
    
    try:
        response = requests.get(f"https://api.thenewsapi.com/v1/news/all?{last_page_params}")
        last_page_data = json.loads(response.content.decode('utf-8'))
        
        print(json.dumps(last_page_data, indent=4))
        if last_page_data['data']:
            furthest_date = last_page_data['data'][-1]['published_at']
            print(f"Furthest (oldest) news date: {furthest_date}")
    except Exception as e:
        print(f"Error fetching last page: {e}")
    
    # STEP 3: Fetch all articles (from cell 3)
    all_articles = []
    all_articles.extend(parsed_data['data'])
    
    with tqdm(total=pages_to_fetch, leave=True, position=0) as pbar:
        for page in range(1, pages_to_fetch + 1):
            pbar.desc = f"Fetching page {page}"
            
            page_params = urllib.parse.urlencode({
                'api_token': api_token,
                'search': search_topic,
                'domains': source_domain,
                'limit': limit,
                'page': page
            })
            
            try:
                response = requests.get(f"https://api.thenewsapi.com/v1/news/all?{page_params}")
                raw_content = response.content
                content = raw_content.decode('utf-8')
                parsed_data = json.loads(content)
                
                all_articles.extend(parsed_data['data'])
                
                time.sleep(0.5)
                
            except Exception as e:
                print(f"Error on page {page}: {e}")
                continue
            
            pbar.update(1)
    
    # STEP 4: Create DataFrame (from cell 4)
    df = pd.DataFrame(all_articles)
    df.sort_values(by='published_at', inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [ ]:
# Fetch CNN crime articles
cnn_df = fetch_news_articles(
    api_token=api_token,
    search_topic='crime',
    source_domain='cnn.com',
    limit=25
)
fox_df = fetch_news_articles(
    api_token=api_token,
    search_topic='crime',
    source_domain='foxnews.com',
    limit=25
)

{
    "meta": {
        "found": 3505,
        "returned": 25,
        "limit": 25,
        "page": 1
    },
    "data": [
        {
            "uuid": "8ff40aee-6754-4363-8d68-2ba77ea03dcf",
            "title": "DC crime law: Why Joe Biden is playing defense on crime",
            "description": "The Senate this week passed a Republican-led resolution to overturn a Washington, DC, crime law, which critics have argued is soft on violent criminals.",
            "keywords": "2020 presidential election, brand safety-nsf crime, brand safety-nsf sensitive, continents and regions, crime, law enforcement and corrections, criminal offenses, domestic alerts, domestic-2020 elections, domestic-us news, domestic-us politics, elections (by type), elections and campaigns, government and public administration, iab-crime, iab-elections, iab-politics, international alerts, international-us 2020 elections, international-us news, international-us politics, joe biden, new york (state), new york city, n

Fetching page 141: 100%|██████████| 141/141 [02:26<00:00,  1.04s/it]


In [ ]:
import os

output_dir = 'data/news'
os.makedirs(output_dir, exist_ok=True)

cnn_df.to_csv('data/news/all_articles_cnn.csv', index=False)
fox_df.to_csv('data/news/all_articles_foxnews.csv', index=False)
# Concatenate CNN and Fox News dataframes
news_df = pd.concat([cnn_df, fox_df], ignore_index=True)

# Create date column in yyyy/mm format from published_at
news_df['date'] = pd.to_datetime(news_df['published_at']).dt.strftime('%Y/%m')

news_df.sort_values(by='date', inplace=True)
news_df.reset_index(drop=True, inplace=True)
news_df.to_csv('data/news/all_articles_combined.csv', index=False)

NameError: name 'fox_df' is not defined